# 5. Processing the smartphone data in real time and sending back to the server

In this tutorial, the data acquired by **intdash Motion** is processed by the **intdash SDK for Python**  (hereinafter referred to as intdash SDK) and uploaded to intdash. As a sample of function processing, the moving average is applied to the acceleration data.

In this case, we will focus on the following methods.

- Retrieve real time data from another edge
- Apply a function to the retrieved data
- Upload the processed data in real time


## 5.0 Preparation

Before starting this scenario, prepare the following.

- An edge that performs measurement
- intdash Motion Application
- Signal definition for general sensor data

### Data to be used
In this scenario, the following data needs to be ready on the server side.

| Data item | Data name that appears in this scenario |
|:---|:---|
|Edge for data acquisition|edge1|
|Edge for data upload|edge2|
| Signal definitions(\*) | `sp_ACCX`, `sp_ACCY`, `sp_ACCZ`|

(\*) Same as the signal definition used in SDK tutorial [3. Saving data from the smartphone as CSV](../3_save-data-as-csv/3_save-data-as-csv.ipynb).


### Import packages and create a client
For `url` given to `intdash.Client`, specify the environment of the intdash server. For `username` and `password`, specify the auth information issued for the edge you use.

In [4]:
import pandas as pd

import intdash
from intdash import timeutils

# Create client
client = intdash.Client(
    url = "https://example.intdash.jp",
    username = "edge1",
    password="password_here"
)

### Confirm that the signal definitions are registered
Confirm that the signal definitions for this scenario are registered.  
If it is not registered, see **(Option) Register the signal definitions** in the next step.

In [48]:
signals = client.signals.list(label='sp')

In [49]:
for s in signals:
    print(s.label,  end=', ')

sp_ACCX, sp_ACCY, sp_ACCZ, 

### (Option) Register the signal definitions
```
Warning:
  If the target signal definition has already been registered to the server, skip this step.
```
Use the same signal definition that you used in the SDK tutorial [3. Saving data from the smartphone as CSV](../3_save-data-as-csv/3_save-data-as-csv.ipynb).
Execute the following file to register the signal definitions.

[0_create-signal-general-sensor.ipynb](../0_create-signal-general-sensor/0_create-signal-general-sensor.ipynb)  

Register only the signal definitions for "acceleration".

## 5.1 Retrieve the edge

In [5]:
edge1 = client.edges.list(name='edge1')[0]
edge2 = client.edges.list(name='edge2')[0]

In [6]:
edge1.name, edge2.name

('edge1', 'edge2')

## 5.2 Create a Queue
The process from download to upload is performed using a Queue.

In [8]:
import queue

q = queue.Queue(maxsize=5)

## 5.3 Preparation of data retrieval (Downstream)
Define a process to receive time series data from an edge through the server.

### 5.3.1 Create a request
For ``src_edge_uuid``, specify the source edge. In this example, the edge ``edge1`` running intdash Motion is used.
Specify the ``label`` name of the signal definition in `data_id` of `core.DataFilter`.

In [9]:
d_specs = [
        intdash.DownstreamSpec(
            src_edge_uuid = edge1.uuid, 
            filters = [
                 intdash.DataFilter(data_type=intdash.DataType.float.value, data_id='sp_ACCX',channel=1),  # Acceleration
                 intdash.DataFilter(data_type=intdash.DataType.float.value, data_id='sp_ACCY',channel=1),  # Acceleration
                 intdash.DataFilter(data_type=intdash.DataType.float.value, data_id='sp_ACCZ',channel=1),  # Acceleration
            ],
        ),
    ]

### 5.3.2 Define a function to be applied to the received data
In this scenario, the data receiving side receives the time series data and adds it to the Queue as it is.

In [10]:
# Put the received time-series data to the Queue.
def callback(unit):
    try:
        q.put_nowait(unit)
    except queue.Full:
        pass

## 5.4 Preparation for data upload (Upstream)
Define a function that processes the received data and sends the data back to the server as new time series data.

### 5.4.1 Create a request
Specify the UUID of the uploading edge.

In [11]:
u_specs = [
        intdash.UpstreamSpec(
            src_edge_uuid = edge2.uuid,
        ),
    ]

### 5.4.2 Define the process
In this scenario, define the process that calculates the moving average and returns it to the server.

In [12]:
import numpy as np

# The function to calculate moving average.
def calc_ave(score, array, ave_num):
    array.append(score)
    if len(array) > ave_num:
        array.popleft()
  
    return  np.sum(array)/ len(array)

### 5.4.3 Define an uploader function

Define the following processes:

- Get data from the Queue
- Switch the process depending on the data type of time series data
- Put data into the process
- Send the newly created data back (return the intdash.Unit with "yield")

In [13]:
AVE_NUM = 5

import struct
from collections import deque

acc_x_dq = deque([])
acc_y_dq = deque([])
acc_z_dq = deque([])


# Calculate moving average of the received time-series data, convert it to Unit and upload. 
def upload_func():
    while True:
        try:
            unit = q.get_nowait()
            
            # Skip basetime.
            if unit.data.data_type.value == intdash.DataType.basetime.value:
                yield unit
                continue
                
            if unit.data.data_type.value != intdash.DataType.float.value:
                yield unit
                continue
                
            # Get intdash.intdash.data.GeneralSensor.
            sensor_data = unit.data
                
            if unit.data.data_id == 'sp_ACCX':
                acc_x = unit.data.value
                ave_acc_x = calc_ave(acc_x, acc_x_dq, AVE_NUM)
                
                if ave_acc_x is None:
                    continue
               
                yield intdash.Unit(
                      elapsed_time = unit.elapsed_time,
                      channel = 1,
                      data =  intdash.data.Float(data_id='sp_ACCX', value=ave_acc_x ),
                    )
                continue
                
                 
            if unit.data.data_id == 'sp_ACCY':
                acc_y = unit.data.value
                ave_acc_y = calc_ave(acc_y, acc_y_dq, AVE_NUM)
                
                if ave_acc_y is None:
                    continue
               
                yield intdash.Unit(
                      elapsed_time = unit.elapsed_time,
                      channel = 1,
                      data =  intdash.data.Float(data_id='sp_ACCY', value=ave_acc_y ),
                    )
                continue
                
            if unit.data.data_id == 'sp_ACCZ':
                acc_z = unit.data.value
                ave_acc_z = calc_ave(acc_z, acc_z_dq, AVE_NUM)
                
                if ave_acc_z is None:
                    continue
                
                yield intdash.Unit(
                      elapsed_time = unit.elapsed_time,
                      channel = 1,
                      data = intdash.data.Float(data_id='sp_ACCZ', value=ave_acc_z ),
                    )
                
                continue
                
        except queue.Empty:
            yield

## 5.5 Start stream processing

In [18]:
wsconn = client.connect_websocket()

### 5.5.1 Start upstream

In [19]:
wsconn.open_upstreams(
    specs = u_specs,
    iterators = [upload_func()],
)

### 5.5.2 Start downstream

In [20]:
wsconn.open_downstreams(
    specs = d_specs,
    callbacks = [callback],
)

## 5.6 Check data with Visual M2M Data Visualizer
By using **Visual M2M Data Visualizer**, you can see that the data is being communicated in real time. If you import the "SCREEN file (.scrn)" and "DAT file (.dat)" stored in the same directory as this notebook into **Visual M2M Data Visualizer**, you can see the data as follows. See **"Setting up Data Visualizer" in intdash tutorial 2**.
 
 
On the screen below, `Acceleration raw` panel shows the data before conversion (data sent by Motion app), and `Acceleration Converted` panel shows the moving average calculated using `intdash-py`.

<img src="https://user-images.githubusercontent.com/70192465/94385047-10d80900-017f-11eb-9c00-aae106b49aaa.png">

## 5.7 Disconnecting real-time processing
If you want to end the process, be sure to execute the following to disconnect.

In [21]:
wsconn.close()